In [1]:
from cassandra.cluster import Cluster

In [2]:
cluster = Cluster(contact_points=['127.0.0.1'])
session = cluster.connect()

In [3]:
session.execute('SELECT * FROM music_library')

InvalidRequest: Error from server: code=2200 [Invalid query] message="No keyspace has been specified. USE a keyspace, or explicitly specify keyspace.tablename"

In [5]:
# Create a keyspace - schema in terms of RDBMS
# Replication means copying the data across nodes, but we are using one node so
# replication_factor = 1
session.execute('''
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = {
    'class': 'SimpleStrategy',
    'replication_factor': 1
    }
''')

In [6]:
# Connect to keyspace
session.set_keyspace('udacity')

In [7]:
# Create a table with (year, artist_name) as primary key
session.execute('''
    CREATE TABLE IF NOT EXISTS music_library
    (year int, artist_name text, album_name text, PRIMARY KEY (year, artist_name))
    ''')

In [10]:
# Check if table exists by select count(*)
# NOTE! Normally we wouldn't do this if we have large data across many nodes
# Cassandra is not for that
count = session.execute('''
    SELECT COUNT(*) from music_library
    ''')
print(count.one())

Row(count=0)


In [11]:
# Insert some values into a table
# Cassandra does not allow duplicates on PRIMARY_KEY, so running this statement again and again
# will not result in duplicates
query = '''
    INSERT INTO music_library (year, artist_name, album_name)
    VALUES (%s, %s, %s)
'''
session.execute(query, parameters=(1970, 'The Beatles', 'Let It Be'))
session.execute(query, parameters=(1965, 'The Beatles', 'Rubber Soul'))

In [12]:
# Select rows with year = 1970
query = 'select * from music_library where year = 1970'
rows = session.execute(query)
for row in rows:
    print (row.year, row.album_name, row.artist_name)

1970 Let It Be The Beatles


In [13]:
# Drop the table and close connection
query = 'drop table music_library'
session.execute(query)
session.shutdown()
cluster.shutdown()